This notebook contains code to test and generate images for the tutorial.

In [ ]:
from IPython.display import Image, SVG, display
import cv2
import smashcima as sc
import numpy as np

In [ ]:
class MyGlyphSynthesizer(sc.GlyphSynthesizer):
    def supports_label(self, label: str) -> bool:
        return # true if we can create this glyph type #

    def create_glyph(self, label: str) -> sc.Glyph:
        return # create requested glyph here #

In [ ]:
print(sc.SmuflLabels.noteheadWhole.value) # "smufl::noteheadWhole"
print(sc.SmuflLabels.noteheadBlack.value) # "smufl::noteheadBlack"
print(sc.SmuflLabels.restQuarter.value)   # "smufl::restQuarter"

print(sc.SmashcimaLabels.beam.value) # "smashcima::beam"
print(sc.SmashcimaLabels.staffMeasure.value) # "smashcima::staffMeasure"

In [ ]:
space = sc.AffineSpace()

size = int(sc.mm_to_px(3, dpi=300))
bitmap = np.zeros(shape=(size, size, 4), dtype=np.uint8)
cv2.circle(
    img=bitmap,
    center=(size // 2, size // 2),
    radius=size // 2,
    color=(0, 0, 255, 255), # BGRA
    thickness=-1
)

# display(Image(
#     data=cv2.imencode(".png", bitmap)[1],
# ))

sprite = sc.Sprite(
    space=space,
    bitmap=bitmap,
    bitmap_origin=sc.Point(0.5, 0.5),
    dpi=300,
    transform=sc.Transform.identity()
)

region: sc.LabeledRegion = sc.Glyph.build_region_from_sprites_alpha_channel(
    label="smufl::noteheadBlack",
    sprites=[sprite]
)

assert region.space is sprite.space # succeeds!
assert region.space is space # succeeds!

In [ ]:
class RedCircleGlyphSynth(sc.GlyphSynthesizer):
    def supports_label(self, label: str) -> bool:
        return True

    def create_glyph(self, label: str) -> sc.Glyph:
        space = sc.AffineSpace()

        size = int(sc.mm_to_px(3, dpi=300))
        bitmap = np.zeros(shape=(size, size, 4), dtype=np.uint8)
        cv2.circle(
            img=bitmap,
            center=(size // 2, size // 2),
            radius=size // 2,
            color=(0, 0, 255, 255), # BGRA
            thickness=-1
        )

        sprite = sc.Sprite(
            space=space,
            bitmap=bitmap,
            bitmap_origin=sc.Point(0.5, 0.5),
            dpi=300,
            transform=sc.Transform.identity()
        )

        return sc.Glyph(
            space=space,
            region=sc.Glyph.build_region_from_sprites_alpha_channel(
                label=label,
                sprites=[sprite]
            ),
            sprites=[sprite]
        )

In [ ]:
class RedCircleModel(sc.orchestration.BaseHandwrittenModel):
    def register_services(self):
        super().register_services()

        # register a different implementation for the interface        
        self.container.interface(
            sc.GlyphSynthesizer,
            RedCircleGlyphSynth
        )

In [ ]:
model = RedCircleModel()
scene = model("../../docs/assets/tutorial-1/lc5003150.musicxml")

bitmap = scene.render(scene.pages[0])
display(Image(
    data=cv2.imencode(".png", bitmap)[1],
    width=900
))